# SPR 2026 - SGDClassifier v4 (RandomSearch Intensivo + SMOTE)

**Score v3:** 0.77036 (+2.7% melhoria!)

**Estratégia v4:** Replicar e intensificar o sucesso do v3
- RandomizedSearchCV com **50 iterações** (vs 30 no v3)
- **SMOTE ativo** para classes 5/6 (500 samples cada)
- Class weights balanceados
- Threshold tuning por classe

**Meta:** Alcançar 0.80+ F1-Macro

---
**CONFIGURAÇÃO KAGGLE:** Internet OFF
---

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.stats import loguniform, uniform
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - SGDClassifier v4 (RandomSearch Intensivo + SMOTE)")
print("="*60)

SEED = 42
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'

# ========== VERIFICAR DATASET PRIMEIRO ==========
if not os.path.exists(DATA_DIR):
    print("\n" + "="*60)
    print("ERRO: Dataset não encontrado!")
    print("="*60)
    print("\nAdicione o dataset:")
    print("Add Input → Competition → spr-2026-mammography-report-classification")
    raise FileNotFoundError(f"Dataset não encontrado: {DATA_DIR}")
print(f"Dataset: {DATA_DIR}")

np.random.seed(SEED)

# ========== FLAGS - v4 CONFIGURAÇÃO ==========
REMOVE_CLASS_2 = False
USE_SMOTE = True                # ATIVO - diferencial v4
USE_THRESHOLD_TUNING = True
N_SEARCH_ITER = 50              # 50 iter (vs 30 no v3)
SMOTE_TARGET_5 = 500            # Oversample classe 5
SMOTE_TARGET_6 = 500            # Oversample classe 6
# =============================================

# Dados
print("\n[1/6] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f"Train: {train.shape} | Test: {test.shape}")

# Auto-detectar colunas
def find_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
        for col in df.columns:
            if col.lower() == c.lower():
                return col
    return None

TEXT_COL = find_col(train, ['report', 'text', 'laudo', 'texto', 'content'])
LABEL_COL = find_col(train, ['target', 'label', 'birads', 'classe', 'class'])
ID_COL = find_col(test, ['ID', 'id', 'Id', 'index', 'idx'])
print(f"Colunas: texto={TEXT_COL}, label={LABEL_COL}, id={ID_COL}")

if REMOVE_CLASS_2:
    train = train[train[LABEL_COL] != 2].reset_index(drop=True)
    print(f"Sem classe 2: {train.shape}")

# Distribuição de classes
print("\nDistribuição de classes:")
print(train[LABEL_COL].value_counts().sort_index())

In [ ]:
# TF-IDF otimizado
print("\n[2/6] TF-IDF...")
tfidf = TfidfVectorizer(
    max_features=20000,          # Aumentado para capturar mais features
    ngram_range=(1, 3),          # Trigramas incluídos
    min_df=2, 
    max_df=0.95, 
    sublinear_tf=True
)
X_train = tfidf.fit_transform(train[TEXT_COL])
X_test = tfidf.transform(test[TEXT_COL])
y_train = train[LABEL_COL].values
print(f"Shape TF-IDF: {X_train.shape}")

In [ ]:
# SMOTE para classes minoritárias
print("\n[3/6] SMOTE para classes 5 e 6...")

if USE_SMOTE:
    try:
        from imblearn.over_sampling import SMOTE
        
        # Verificar contagem atual
        unique, counts = np.unique(y_train, return_counts=True)
        print("Antes do SMOTE:")
        for u, c in zip(unique, counts):
            print(f"  Classe {u}: {c} samples")
        
        # SMOTE apenas para classes 5 e 6
        smote_strategy = {}
        if 5 in unique and counts[list(unique).index(5)] < SMOTE_TARGET_5:
            smote_strategy[5] = SMOTE_TARGET_5
        if 6 in unique and counts[list(unique).index(6)] < SMOTE_TARGET_6:
            smote_strategy[6] = SMOTE_TARGET_6
        
        if smote_strategy:
            smote = SMOTE(
                sampling_strategy=smote_strategy, 
                random_state=SEED, 
                k_neighbors=3
            )
            X_train, y_train = smote.fit_resample(X_train, y_train)
            
            print(f"\nDepois do SMOTE: {X_train.shape}")
            unique, counts = np.unique(y_train, return_counts=True)
            for u, c in zip(unique, counts):
                print(f"  Classe {u}: {c} samples")
        else:
            print("SMOTE não necessário - classes já têm samples suficientes")
            
    except ImportError:
        print("⚠️ imblearn não disponível - continuando sem SMOTE")
else:
    print("SMOTE desativado")

In [ ]:
# RandomizedSearch INTENSIVO
print(f"\n[4/6] RandomizedSearchCV INTENSIVO ({N_SEARCH_ITER} iterações)...")

param_dist = {
    'alpha': loguniform(1e-6, 1e-2),           # Range mais amplo
    'loss': ['log_loss', 'modified_huber'],     # Focado nos que funcionam
    'penalty': ['l2', 'l1', 'elasticnet'],
    'l1_ratio': uniform(0.1, 0.8),              # Distribuição contínua
    'max_iter': [2000, 3000, 5000],              # Mais iterações
    'learning_rate': ['optimal', 'adaptive'],
    'eta0': loguniform(1e-4, 1e-1),
    'power_t': uniform(0.1, 0.9),                # Novo parâmetro
    'tol': loguniform(1e-5, 1e-3),              # Tolerância
}

base = SGDClassifier(
    class_weight='balanced', 
    random_state=SEED, 
    n_jobs=-1, 
    early_stopping=True, 
    validation_fraction=0.1,
    shuffle=True
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

search = RandomizedSearchCV(
    base, 
    param_dist, 
    n_iter=N_SEARCH_ITER, 
    cv=cv, 
    scoring='f1_macro', 
    n_jobs=-1, 
    random_state=SEED, 
    verbose=2,
    return_train_score=True
)

search.fit(X_train, y_train)

print(f"\n{'='*60}")
print(f"MELHORES PARÂMETROS:")
print(f"{'='*60}")
for param, value in search.best_params_.items():
    print(f"  {param}: {value}")
print(f"\nMelhor F1-macro CV: {search.best_score_:.5f}")
print(f"{'='*60}")

best_model = search.best_estimator_

In [ ]:
# Threshold tuning por classe
print("\n[5/6] Threshold tuning...")

if USE_THRESHOLD_TUNING and hasattr(best_model, 'predict_proba'):
    proba = best_model.predict_proba(X_test)
    classes = best_model.classes_
    
    # Thresholds ajustados para classes minoritárias
    thresholds = {
        0: 0.50,
        1: 0.50,
        2: 0.50,
        3: 0.50,
        4: 0.50,
        5: 0.30,  # Mais sensível - classe minoritária
        6: 0.25   # Muito mais sensível - classe muito minoritária
    }
    
    print("Thresholds por classe:")
    for c, t in thresholds.items():
        print(f"  Classe {c}: {t}")
    
    preds = []
    for i in range(len(proba)):
        adj = proba[i].copy()
        for j, c in enumerate(classes):
            if c in thresholds:
                # Ajustar probabilidades baseado no threshold
                adj[j] *= (0.5 / thresholds[c])
        preds.append(classes[np.argmax(adj)])
    
    predictions = np.array(preds)
else:
    print("Predição padrão (sem threshold tuning)...")
    predictions = best_model.predict(X_test)

In [ ]:
# Submissão
print("\n[6/6] Gerando submissão...")

sample_path = f'{DATA_DIR}/sample_submission.csv'
if os.path.exists(sample_path):
    sample_sub = pd.read_csv(sample_path)
    SUB_ID = sample_sub.columns[0]
    SUB_LABEL = sample_sub.columns[1]
else:
    SUB_ID = ID_COL
    SUB_LABEL = LABEL_COL

submission = pd.DataFrame({SUB_ID: test[ID_COL], SUB_LABEL: predictions})
submission.to_csv('/kaggle/working/submission.csv', index=False)

print("="*60)
print("SGDClassifier v4 CONCLUÍDO!")
print("="*60)
print(f"\nMelhor F1-macro CV: {search.best_score_:.5f}")
print("\nDistribuição das predições:")
print(submission[SUB_LABEL].value_counts().sort_index())
print("\n✅ submission.csv criado!")